In [11]:
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
import pandas as pd

#from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

### Neo4j Credentials

In [13]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [18]:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

### Connecting to Neo4j

In [19]:
URI = NEO4J_URI
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

In [21]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [14]:
raw_data = pd.read_csv('./data/char_relations.csv')
raw_data.head()

,name,friend,classmate,teacher,family,immediate family,lover,opponent,colleague,teammate,enemy,"Harry's affection to him (-10 to 10, 0 is the initial value)","Harry's familiarity with him (0 to 10, 0 is the initial value)","His affection to Harry (0 to 10, 0 is the initial value)","His familiarity with Harry (0 to 10, 0 is the initial value)"
0,James,0,0,0,1,1,0,0,0,0,0,0,0,10,0
1,Lily,0,0,0,1,1,0,0,0,0,0,0,0,10,0
2,Petunia,0,0,0,1,0,0,0,0,0,0,0,0,-2,1
3,Vernon Dursley,0,0,0,1,0,0,0,0,0,0,0,0,-2,1
4,Dudley,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
raw_data.describe()

,friend,classmate,teacher,family,immediate family,lover,opponent,colleague,teammate,enemy,"Harry's affection to him (-10 to 10, 0 is the initial value)","Harry's familiarity with him (0 to 10, 0 is the initial value)","His affection to Harry (0 to 10, 0 is the initial value)","His familiarity with Harry (0 to 10, 0 is the initial value)"
count,31.0,31.0,31.0,31.000000,31.000000,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.000000,31.000000
mean,0.0,0.0,0.0,0.129032,0.064516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.580645,0.580645
std,0.0,0.0,0.0,0.340777,0.249731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.393859,1.360740
min,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.000000,0.000000
25%,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
50%,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
75%,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
max,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.000000,4.000000


### Creating Knowledge Graph

In [22]:
def create_characters(tx, name):
    tx.run("MERGE (p:Person {name: $name})", name=name)

In [23]:
def create_relationship(tx, person1, relationship, person2, attributes):
    tx.run(f"MATCH (a:Person {{name: $person1}}), (b:Person {{name: $person2}}) "
           f"MERGE (a)-[r:{relationship.upper()}]->(b) "
           f"SET r += $attributes",
           person1=person1, person2=person2, attributes=attributes)


In [29]:
with driver.session() as session:
    with open('./data/char_relations.csv', 'r') as file:
        
        session.write_transaction(create_characters, 'Harry')
        
        for line in file:
            next(file)
            
            name, friend, classmate, teacher, family, immediate, lover, opponent, colleague, teammate, enemy, harry_aff, harry_familiar, his_affection, his_familiar = line.strip().split(',')

            # Create nodes
            session.write_transaction(create_characters, name)

            # Create relationships
            if family == '1':
                session.write_transaction(create_relationship, 'Harry', "FAMILY", name, {})
            if lover == '1':
                session.write_transaction(create_relationship, 'Harry', "LOVER", name, {})

            # Set attributes 
            session.run(f"MATCH (p:Person {{name: $name}}) "
                        f"SET p.harry_aff = $harry_aff, p.his_affection = $his_affection",
                        name=name, harry_aff=int(harry_aff), his_affection=int(his_affection))
            
            session.run(f"MATCH (p:Person {{name: $name}}) "
                        f"SET p.harry_familiar = $harry_familiar, p.his_familiar = $his_familiar",
                        name=name, harry_aff=int(harry_aff), his_affection=int(his_affection))

driver.close()

name


ValueError: invalid literal for int() with base 10: "Harry's affection to him (-10 to 10)"

In [ ]:
def insert_data(tx, source, target, relationship):
    query = (
        "MERGE (a:Person {name: $source}) "
        "MERGE (b:Person {name: $target}) "
        f"MERGE (a)-[:{relationship}]->(b)"
    )
    
    tx.run(query, source=source, target=target)
